# 15.1 자산 가격결정 기본정리
## 15.1.2 일반적인 결과

이산 시간에서 일반 시장 모형 M
 - 유한 상태 공간 $\Omega$
 - 필트레이션 F
 - $\Omega$에서 정의된 0이 아닌 확률 측도 P
 - 만기 시간 T $∈$ N, T < $∞$
 - K+1개의 증권에 대한, 양의 가격 프로세스 집합 $S = {｛S^K_t｝_{t∈｛0,...T｝} : k∈｛0,....,K｝}$

$M = ｛(\Omega, \gamma(\Omega),F,P),T,S｝$

일반 시장 모형 M조건에서 자산 가격결정 기본 정리에 따르면 다음은 모두 동치
 - 시장 모형에서 차익거래 기회가 존재하지 않는다.
 - P-동치 마틴게일 측도 집합 Q의 원소가 존재한다.
 - consistent linear price system의 집합 P의 원소가 존재한다.

시장 모형 M에 차익거래 기회가 없으면 (옵션, 기타 파생상품 등) 어떠한 종류의 복제 가능한 조건부 청구권 $V_T$에 대해서도 유일한 가격 $V_0$가 존재한다.

$∀Q ∈ Q : V_0 = E^Q_0(e^{-rT}V_T)$

IF 시장 모형 M에 차익거래 기회가 없고 모든 조건부 청구권이 복제 가능하다면, 즉 그 시장 모형이 완전하다면 Q의 원소가 하나 밖에 없는 경우와 동치이다.( 하나의 유일한 P-동치 마틴게일 측도만이 존재) 
 

# 15.2 위험 중립 할인
## 15.2.1 날짜 모형과 조작

In [2]:
import datetime as dt
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7, 1), dt.datetime(2016, 1, 1)]

In [6]:
# 연 비율
(dates[1] - dates[0]).days/365.

0.4958904109589041

In [4]:
(dates[2] - dates[1]).days/365.

0.5041095890410959

In [5]:
fractions = [0.0, 0.5, 1.0]

In [7]:
import numpy as np

def get_year_deltas(date_list, day_count = 365.):
    ''' 날짜를 1년 비율로 표현한 부동소수점 벡터를 전환.
    초깃값은 0으로 고정
    
    인수
    ====
    date_list : list or array
        datetime 객체 모음
    
    day_count : float
        1년의 날짜 수
    
    반환값
    ======
    delta_list : array
        1년 기준 비율
    '''
    
    start = date_list[0]
    delta_list = [(date - start).days / day_count for date in date_list]
    return np.array(delta_list)

In [8]:
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7, 1), dt.datetime(2016, 1, 1)]

In [9]:
get_year_deltas(dates)

array([0.        , 0.49589041, 1.        ])

# 15.2.2 고정 단기 이자율

In [11]:
# from get_year_deltas import * 
class constant_short_rate(object):
    ''' 고정 단기 이자율 할인 클래스
    
    속성
    ====
    name : string
        객체 이름
    short_rate : float (positive)
        할인에 사용될 고정 이자율
    
    Method
    =======
    get_disocunt_factors :
        datetime 객체 or 1년 기준 비율의 리스트/배열이 주어지면 할인율 반환
    '''
    
    def __init__(self, name, short_rate):
        self.name = name
        self.short_rate = short_rate
        if short_rate < 0:
            raise ValueError('Short rate negative.')
        
    def get_discount_factors(self, date_list, dtobjects = True):
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        
        else:
            dlist = np.array(date_list)
        
        dflist = np.exp(self.short_rate * np.sort(-dlist))
        return np.array((date_list, dflist)).T

In [13]:
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7, 1), dt.datetime(2016, 1, 1)]
# from constant_short_rate import *
csr = constant_short_rate('csr', 0.05)
csr.get_discount_factors(dates)

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.951229424500714],
       [datetime.datetime(2015, 7, 1, 0, 0), 0.9755103387657228],
       [datetime.datetime(2016, 1, 1, 0, 0), 1.0]], dtype=object)

In [18]:
deltas = get_year_deltas(dates)
csr.get_discount_factors(deltas, dtobjects = False)

array([[0.        , 0.95122942],
       [0.49589041, 0.97551034],
       [1.        , 1.        ]])

# 15.3 시장 환경

In [21]:
class market_environment(object):
    ''' 가치 평가와 관련된 시장 환경 모형을 위한 클래스
    
    속성
    ====
    name : string
        시장 환경 이름
    pricing_date : datetime object
        시장 환경 날짜
    
    Method
    =======
    add_constant :
        상수(모형 파라미터) 추가
    get_constant :
        상수 반환
    add_list :
        (기초자산 등의) 리스트 추가
    get_list :
        리스트 반환
    add_curve :
        (이자율 커브 등의) 시장 커브 추가
    get_curve :
        시장 커브 반환
    add_environment :
        상수, 리스트, 커브 등의 전체 시장 환경 추가 또는 갱신
    '''
    
    def __init__(self, name, pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
    
    def add_constant(self, key, constant):
        self.constants[key] = constant
    
    def get_constant(self, key):
        return self.constants[key]
    
    def add_list(self, key, list_object):
        self.lists[key] = list_object
    
    def get_list(self, key):
        return self.lists[key]
    
    def add_curve(self, key, curve):
        self.curves[key] = curve
    
    def get_curve(self, key):
        return self.curves[key]
    
    def add_environment(self, env):
        # 존재하는 값이 있을땐 갱신
        for key in env.constants:
            self.constants[key] = env.constants[key]
        
        for key in env.lists:
            self.lists[key] = env.lists[key]
        
        for key in env.curves:
            self.curves[key] = env.curves[key]

In [25]:
# from market_environment import *
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7, 1), dt.datetime(2016, 1, 1)]
csr = constant_short_rate('csr', 0.05)
me_1 = market_environment('me_1', dt.datetime(2015, 1,1))
me_1.add_list('symbols', ['AAPL', 'MSFT', 'FB'])
me_1.get_list('symbols')

['AAPL', 'MSFT', 'FB']

In [26]:
me_2 = market_environment('me_2', dt.datetime(2015, 1, 1))
me_2.add_constant('volatility', 0.2)
me_2.add_curve('short_rate', csr)
me_2.get_curve('short_rate')

In [27]:
me_1.add_environment(me_2)
me_1.get_curve('short_rate')

In [28]:
me_1.constants

{'volatility': 0.2}

In [29]:
me_1.lists

{'symbols': ['AAPL', 'MSFT', 'FB']}

In [30]:
me_1.curves

{'short_rate': <__main__.constant_short_rate at 0x291aa71d788>}

In [31]:
me_1.get_curve('short_rate').short_rate

0.05